In [ ]:
# 여러명의 데이터를 리스트로 입력받아 처리
# 성적에 대한 CRUD
# 학생에 대한 CRUD
# 통계에 대한 CRUD 등을 작성한다
# 함수를 활용해서 학생의 석차를 구하는 함수

import os

# 성적을 등급으로 변환하는 함수
def get_grade(score):
    if score >= 90:
        return 'A'
    elif score >= 80:
        return 'B'
    elif score >= 70:
        return 'C'
    elif score >= 60:
        return 'D'
    else:
        return 'F'

# 성적을 등수로 변환하는 함수
def get_rank(score, scores):
    rank = 1
    for s in scores:
        if score < s:
            rank += 1
    return rank

# 동석차 학생 수 구하는 함수
def get_tied_rank_count(score, scores):
    tied_count = scores.count(score)
    return tied_count  # 동석차 학생 수를 반환

name, kor, eng, math, py = [],[],[],[],[]
subjects = [kor, eng, math, py]
subject_names = ["국어", "영어", "수학", "파이썬"]
total_scores = []
error_count = 0

# 초기값 설정
initial_data = [
    ("김철수", [85, 90, 80, 70]),
    ("이영희", [85, 92, 75, 85]),
    ("박민수", [78, 85, 85, 95]),
    ("정다은", [90, 85, 90, 92]),
    ("최준호", [75, 65, 90, 85])
]

# 성적표.txt 초기 데이터로 리셋
file_path = "성적표.txt"
#if not os.path.exists(file_path) or os.stat(file_path).st_size == 0:
with open(file_path, "w", encoding="utf-8") as file:
    for input_name, scores in initial_data:
        file.write(f"{input_name}, {', '.join(map(str, scores))}, {sum(scores)}\n")

# 초기값을 리스트에 추가
for input_name, scores in initial_data:
    if input_name not in name:
        name.append(input_name)
        for i, temp_score in enumerate(scores):
            subjects[i].append(temp_score)
        total_scores.append(sum(scores))


run = True
confirm = ""
prompt = """1. 이름/성적 입력
2. 성적 확인
3. 성적 수정
4. 성적 삭제(초기화)
5. 이름 수정/삭제
6. 프로그램 종료

Enter number(1~6): """

while run:
    print(prompt)
    select = input("번호 입력: ")

    if select == "1":
        print("이름 및 성적 입력\n")
        input_name = input("이름: ").strip()    # 좌우 공백 제거
        input_name = input_name.replace(" ", "")  # 모든 공백 제거
        if input_name in name:
            print("이미 등록된 이름입니다.")
            continue

        # 성적 입력을 위한 리스트 초기화
        temp_subjects = []
        valid_input = True  # 모든 입력이 유효한지 확인하는 플래그

        for i, subject in enumerate(subjects):
            try:
                score = int(input(f"{subject_names[i]}: "))
                if 0 <= score <= 100:  # 점수가 0~100 사이인지 확인
                    temp_subjects.append(score)
                else:
                    print("점수는 0에서 100 사이여야 합니다.")
                    valid_input = False
                    break  # 점수 범위 벗어나면 중단
            except ValueError:
                print("잘못된 값을 입력하셨습니다. 숫자를 입력해주세요.")
                valid_input = False
                break  # 숫자가 아니면 중단

        if valid_input:
            name.append(input_name)
            for i, temp_score in enumerate(temp_subjects):
                subjects[i].append(temp_score)  # 성적이 정상적이면 subjects에 추가
            total = sum(temp_subjects)
            total_scores.append(total)
            print("입력되었습니다.\n")

            # 성적을 파일에 추가 (append 모드 사용)
            with open(file_path, "a", encoding="utf-8") as file:
                file.write(f"{input_name}, {', '.join(map(str, temp_subjects))}, {total}\n")

        else:
            print("성적 입력이 취소되었습니다. 다시 시도해주세요.\n")

    elif select == "2":
        print("성적 확인\n")
        search_name = input("이름 입력: ").strip()  # 좌우 공백 제거
        search_name = search_name.replace(" ", "")  # 모든 공백 제거
        found = False  # 이름이 있는지 여부 확인

        with open(file_path, "r", encoding="utf-8") as file:
            for line in file:
                data = line.strip().split(", ")
                student_name = data[0]
                scores = list(map(int, data[1:-1]))  # 마지막 값(총점) 제외하고 점수만 변환
                total = int(data[-1])  # 총점 (마지막 값)

                if search_name == student_name:
                    found = True
                    avg = total / len(scores)  # 평균 계산
                    grade = get_grade(avg)

                    print(f"{student_name}의 성적:")
                    for i, score in enumerate(scores):
                        grade = get_grade(score)
                        print(f"{subject_names[i]}: {score} ({grade})", end="  ")

                    print(f"\n총점: {total}  평균: {avg:.2f} ({grade}) \n")
                    break  # 찾았으면 더 이상 반복할 필요 없음

        if not found:
            print("해당 이름이 존재하지 않습니다.\n")

    elif select == "3":
        print("성적 수정\n")
        search_name = input("이름 입력: ").strip()
        search_name = search_name.replace(" ", "")

        updated_data = []
        found = False

        with open(file_path, "r", encoding="utf-8") as file:
            for line in file:
                data = line.strip().split(", ")
                student_name = data[0]
                if search_name == student_name:
                    found = True
                    print(f"{search_name}의 기존 성적을 수정합니다.")

                    new_scores = []
                    for i, subject in enumerate(subject_names):
                        while True:
                            try:
                                score = int(input(f"{subject}: "))
                                if 0 <= score <= 100:
                                    new_scores.append(score)
                                    break
                                else:
                                    print("점수는 0~100 사이여야 합니다.")
                            except ValueError:
                                print("잘못된 값을 입력하셨습니다. 숫자를 입력해주세요.")

                    total = sum(new_scores)
                    updated_data.append(f"{search_name}, {', '.join(map(str, new_scores))}, {total}\n")
                else:
                    updated_data.append(line)  # 수정 대상이 아니면 그대로 추가

        if found:
            with open(file_path, "w", encoding="utf-8") as file:
                file.writelines(updated_data)
            print("수정이 완료되었습니다.\n")
        else:
            print("해당 이름이 존재하지 않습니다.\n")

    elif select == "4":
        confirm = ""
        print("성적 삭제(초기화)\n")
        while confirm not in ["y", "n"]:
            confirm = input("정말 삭제하시겠습니까? (y/n) ")
            if confirm == 'y':
                search_name = input("이름 입력: ").strip()
                search_name = search_name.replace(" ", "")
                updated_data = []
                found = False

                with open(file_path, "r", encoding="utf-8") as file:
                    for line in file:
                        data = line.strip().split(", ")
                        student_name = data[0]
                        if search_name == student_name:
                            found = True
                            # 성적을 0으로 초기화
                            data[1:] = ["0"] * (len(data) - 1)  # 성적 부분만 "0"으로 바꿈
                        updated_data.append(", ".join(data) + "\n")

                if found:
                    while True:
                        confirm = input(f"정말 {search_name}의 성적을 삭제하시겠습니까? (y/n): ").strip().lower()
                        if confirm == "y":
                            # 파일을 덮어쓰기 모드로 열고 업데이트된 데이터를 쓴다
                            with open(file_path, "w", encoding="utf-8") as file:
                                file.writelines(updated_data)
                            print("성적이 초기화되었습니다.\n")
                            break
                        elif confirm == "n":
                            print("성적 초기화가 취소되었습니다.\n")
                            break
                        else:
                            print("잘못된 입력입니다. 'y' 또는 'n'을 입력해주세요.\n")
                else:
                    print("해당 이름이 존재하지 않습니다.\n")

    elif select == "5":
        confirm = ""
        print("이름 수정/삭제\n")
        name_select = int(input("1. 수정 2. 삭제: "))

        if name_select == 1:  # 이름 수정
            print("이름 수정\n")
            search_name = input("이름 입력: ").strip().replace(" ", "")

            updated_data = []  # 수정된 데이터를 저장할 리스트
            found = False

            with open(file_path, "r", encoding="utf-8") as file:
                for line in file:
                    data = line.strip().split(", ")
                    student_name = data[0]
                    if search_name == student_name:
                        found = True
                        new_name = input("새로운 이름 입력: ").strip().replace(" ", "")
                        data[0] = new_name  # 이름 수정
                    updated_data.append(", ".join(data) + "\n")

            if found:
                with open(file_path, "w", encoding="utf-8") as file:
                    file.writelines(updated_data)
                print("이름이 수정되었습니다.\n")
            else:
                print("해당 이름이 존재하지 않습니다.\n")

        elif name_select == 2:  # 이름 삭제
            print("이름 삭제\n")
            while True:
                confirm = input("정말 삭제하시겠습니까? (y/n): ").strip().lower()
                if confirm == "y":
                    search_name = input("이름 입력: ").strip().replace(" ", "")

                    updated_data = []  # 삭제된 데이터를 제외한 나머지 데이터를 저장할 리스트
                    found = False

                    with open(file_path, "r", encoding="utf-8") as file:
                        for line in file:
                            data = line.strip().split(", ")
                            student_name = data[0]
                            if search_name == student_name:
                                found = True  # 삭제할 학생 찾음
                            else:
                                updated_data.append(line)  # 삭제 대상이 아니면 유지

                    if found:
                        with open(file_path, "w", encoding="utf-8") as file:
                            file.writelines(updated_data)
                        print("삭제되었습니다.\n")
                    else:
                        print("해당 이름이 존재하지 않습니다.\n")
                    break  # 삭제 후 종료

                elif confirm == "n":
                    print("삭제가 취소되었습니다.\n")
                    break  # 취소 후 종료

                else:
                    print("잘못된 입력입니다. 'y' 또는 'n'을 입력해주세요.\n")

    elif select == "6":
        print("프로그램 종료")
        run = False

    else :
        error_count += 1
        print("올바른 값을 입력해주세요.")
        print("오류 3회시 프로그램 강제종료. 현재 %d회" %error_count)
        if error_count >= 3:
            print("프로그램을 종료합니다.")
            run = False


1. 이름/성적 입력
2. 성적 확인
3. 성적 수정
4. 성적 삭제(초기화)
5. 이름 수정/삭제
6. 프로그램 종료

Enter number(1~6): 
번호 입력: 2
성적 확인

이름 입력: 김철수
김철수의 성적:
국어: 85 (B)  영어: 90 (A)  수학: 80 (B)  파이썬: 70 (C)  
총점: 325  평균: 81.25 (C) 

1. 이름/성적 입력
2. 성적 확인
3. 성적 수정
4. 성적 삭제(초기화)
5. 이름 수정/삭제
6. 프로그램 종료

Enter number(1~6): 
번호 입력: 4
성적 삭제(초기화)

정말 삭제하시겠습니까? (y/n) y
이름 입력: 김철수
정말 김철수의 성적을 삭제하시겠습니까? (y/n): y
성적이 초기화되었습니다.

1. 이름/성적 입력
2. 성적 확인
3. 성적 수정
4. 성적 삭제(초기화)
5. 이름 수정/삭제
6. 프로그램 종료

Enter number(1~6): 
번호 입력: 2
성적 확인

이름 입력: 5
해당 이름이 존재하지 않습니다.

1. 이름/성적 입력
2. 성적 확인
3. 성적 수정
4. 성적 삭제(초기화)
5. 이름 수정/삭제
6. 프로그램 종료

Enter number(1~6): 
번호 입력: 5
이름 수정/삭제

1. 수정 2. 삭제: 1
이름 수정

이름 입력: 김철수
새로운 이름 입력: 김
이름이 수정되었습니다.

1. 이름/성적 입력
2. 성적 확인
3. 성적 수정
4. 성적 삭제(초기화)
5. 이름 수정/삭제
6. 프로그램 종료

Enter number(1~6): 
번호 입력: 2
성적 확인

이름 입력: 김
김의 성적:
국어: 0 (F)  영어: 0 (F)  수학: 0 (F)  파이썬: 0 (F)  
총점: 0  평균: 0.00 (F) 

1. 이름/성적 입력
2. 성적 확인
3. 성적 수정
4. 성적 삭제(초기화)
5. 이름 수정/삭제
6. 프로그램 종료

Enter number(1~6): 
